In [10]:
# ANÁLISE FATORIAL

# 1. Instalação e Importação
!pip install factor_analyzer pandas numpy scipy -q

import numpy as np
import pandas as pd
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_kmo, calculate_bartlett_sphericity
from scipy.stats import multivariate_normal
from IPython.display import display, HTML

# 2. Configuração da simulação completa (41 Itens)
np.random.seed(42)
N = 500
total_questions = 41

# Definição dos subgrupos para os índices (16 itens determinantes)
items_isp = ['Q1', 'Q2', 'Q3', 'Q4', 'Q6', 'Q30', 'Q31'] # Itens de Sobrecarga
items_iat = ['Q15', 'Q16', 'Q17', 'Q18', 'Q20', 'Q21', 'Q22'] # Itens de Apoio
items_nsp = ['Q7', 'Q8', 'Q9', 'Q24', 'Q25', 'Q27'] # Itens de Satisfação/Significado
determinants = items_isp + items_iat + items_nsp

# Criar nomes para todas as 41 questões
all_q_names = [f'Q{i}' for i in range(1, total_questions + 1)]

# Construção da Matriz de Correlação para os 41 itens
# Iniciamos com uma matriz identidade (sem correlação)
cor_matrix = np.eye(total_questions)

# Aplicar correlações teóricas nos blocos dos índices (Simulando consistência interna)
for i, items in enumerate([items_isp, items_iat, items_nsp]):
    indices = [all_q_names.index(q) for q in items]
    for idx1 in indices:
        for idx2 in indices:
            if idx1 != idx2:
                cor_matrix[idx1, idx2] = 0.6 # Correlação interna forte

# 3. Geração dos dados (Monte Carlo)
# Geramos dados para todas as 41 questões
means = [3] * total_questions
data_sim = multivariate_normal.rvs(mean=means, cov=cor_matrix, size=N)
df_complete = pd.DataFrame(data_sim, columns=all_q_names)

# Discretização para Escala Likert 1-5
df_complete = df_complete.apply(lambda x: np.clip(np.round(x), 1, 5)).astype(int)

# 4. Execução da Análise Fatorial (apenas nos itens determinantes)
# O modelo científico foca na redução de dimensionalidade dos índices
fa = FactorAnalyzer(n_factors=3, rotation='promax', method='principal')
fa.fit(df_complete[determinants])

# --- FUNÇÃO DE ESTILIZAÇÃO ---
def stylize_table(df, title):
    return df.style.set_caption(f"<b>{title}</b>")\
        .format(precision=3)\
        .map(lambda v: 'font-weight: bold; color: #2E86C1;' if abs(v) >= 0.3 else 'color: #999;')\
        .set_table_styles([
            {'selector': 'caption', 'props': [('color', 'black'), ('font-size', '16px'), ('text-align', 'center'), ('padding', '10px')]},
            {'selector': 'th', 'props': [('background-color', '#f2f2f2'), ('color', 'black'), ('border', '1px solid #ddd')]},
            {'selector': 'td', 'props': [('border', '1px solid #ddd'), ('text-align', 'center')]}
        ])

# --- TABELA 1: MATRIZ DE CARGAS (Apenas Determinantes) ---
loadings_df = pd.DataFrame(fa.loadings_, index=determinants, columns=['Fator ISP', 'Fator IAT', 'Fator NSP'])

# --- TABELA 2: CORRELAÇÃO ENTRE FATORES ---
phi_df = pd.DataFrame(fa.phi_, index=['Índice ISP', 'Índice IAT', 'Índice NSP'], columns=['Índice ISP', 'Índice IAT', 'Índice NSP'])

# --- TABELA 3: ADEQUAÇÃO E CONFIABILIDADE ---
kmo_all, kmo_model = calculate_kmo(df_complete[determinants])
def get_alpha(data):
    corr = data.corr()
    n = data.shape[1]
    avg_corr = corr.values[np.triu_indices_from(corr.values, k=1)].mean()
    return (n * avg_corr) / (1 + (n - 1) * avg_corr)

summary_df = pd.DataFrame({
    'Métrica': ['KMO Global', 'Alfa de Cronbach (ISP)', 'Alfa de Cronbach (IAT)', 'Alfa de Cronbach (NSP)'],
    'Valor': [kmo_model, get_alpha(df_complete[items_isp]), get_alpha(df_complete[items_iat]), get_alpha(df_complete[items_nsp])]
})

# --- EXIBIÇÃO ---
print(f"Dataset completo gerado com {df_complete.shape[1]} colunas.")
display(stylize_table(loadings_df, "Tabela 1: Matriz de Cargas (Itens Determinantes)"))
display(stylize_table(phi_df, "Tabela 2: Matriz de Correlação entre Fatores"))
display(summary_df.style.set_caption("<b>Tabela 3: Consistência Interna</b>").format(subset=['Valor'], precision=3))

# Exportar o dataset completo para download
df_complete.to_csv('dataset_completo_41_itens.csv', index=False)

Dataset completo gerado com 41 colunas.


/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


,Fator ISP,Fator IAT,Fator NSP
Q1,0.011,0.787,0.007
Q2,0.003,0.769,0.024
Q3,-0.042,0.761,0.050
Q4,0.026,0.781,-0.028
Q6,-0.038,0.769,-0.046
Q30,0.066,0.764,-0.010
Q31,-0.022,0.792,-0.002
Q15,0.782,-0.022,-0.067
Q16,0.742,0.065,0.012
Q17,0.784,0.019,0.009


,Índice ISP,Índice IAT,Índice NSP
Índice ISP,1.000,0.018,0.051
Índice IAT,0.018,1.000,-0.015
Índice NSP,0.051,-0.015,1.000


,Métrica,Valor
0,KMO Global,0.908
1,Alfa de Cronbach (ISP),0.889
2,Alfa de Cronbach (IAT),0.888
3,Alfa de Cronbach (NSP),0.892


In [11]:
# BAIXAR ARQUIVOS

# 1. Importação necessária para download no Colab
from google.colab import files
import pandas as pd
import numpy as np

def gerar_e_baixar_arquivos(df_input):
    """
    Gera os CSVs com 41 itens, calcula os índices e aciona o download.
    df_input: DataFrame gerado na célula de simulação (contendo Q1 a Q41).
    """

    # Criar uma cópia para não alterar o original
    df = df_input.copy()

    # --- A. Definição dos Grupos para Cálculo dos Índices ---
    # Conforme definido na estrutura de macro-fatores do COPSOQ-PT Curto
    items_isp = ['Q1', 'Q2', 'Q3', 'Q4', 'Q6', 'Q30', 'Q31']
    items_iat = ['Q15', 'Q16', 'Q17', 'Q18', 'Q20', 'Q21', 'Q22']
    items_nsp = ['Q7', 'Q8', 'Q9', 'Q24', 'Q25', 'Q27']

    # --- B. Cálculo dos Índices (Médias Simples) ---
    df['Indice_ISP'] = df[items_isp].mean(axis=1)
    df['Indice_IAT'] = df[items_iat].mean(axis=1)
    df['Indice_NSP'] = df[items_nsp].mean(axis=1)

    # --- C. Exportar Dataset de Respostas Brutas (41 Itens) ---
    colunas_originais = [f'Q{i}' for i in range(1, 42)]
    df[colunas_originais].to_csv('dataset_simulado_41_itens.csv', index=False)

    # --- D. Exportar Dataset Completo (41 Itens + Índices) ---
    df.to_csv('dataset_com_indices_calculados.csv', index=False)

    # --- E. Gerar Arquivo de Metadados (Documentação) ---
    conteudo_readme = f"""
    PROJETO: Monitoramento de Riscos Psicossociais (Indicadores Integrados)
    DATA DE GERAÇÃO: 2024
    METODOLOGIA: Simulação de Monte Carlo
    ESTRUTURA: 41 Itens (COPSOQ-PT Versão Curta)
    PARAMETROS:
    - Tamanho da Amostra (N): 500
    - Semente Aleatória (Seed): 42
    - Escala: Likert 1-5 (Discretizada)
    - Índices Calculados: ISP (Sobrecarga), IAT (Apoio), NSP (Satisfação)
    """
    with open('metadados_projeto_copsoq.txt', 'w') as f:
        f.write(conteudo_readme)

    # --- F. Acionar o Download no Navegador ---
    print("Iniciando o download dos arquivos do projeto (41 itens)...")

    arquivos = [
        'dataset_simulado_41_itens.csv',
        'dataset_com_indices_calculados.csv',
        'metadados_projeto_copsoq.txt'
    ]

    for arq in arquivos:
        files.download(arq)
        print(f"-> {arq} enviado para o navegador.")

# Chamada da função passando o dataframe 'df_complete' gerado na célula anterior
# Se o seu dataframe tiver outro nome, ajuste aqui:
try:
    gerar_e_baixar_arquivos(df_complete)
except NameError:
    print("Erro: O DataFrame 'df_complete' não foi encontrado. Certifique-se de rodar a célula de simulação primeiro.")

Iniciando o download dos arquivos do projeto (41 itens)...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

-> dataset_simulado_41_itens.csv enviado para o navegador.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

-> dataset_com_indices_calculados.csv enviado para o navegador.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

-> metadados_projeto_copsoq.txt enviado para o navegador.
